This file will learn follow reference https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede

In [1]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    return contents

In [2]:
def individual_preprocess( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence" : [] , "Word" : [] , "Tag" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word[ 0 : word.find('(')] )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
            else:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
    return pre_data_frame , count

In [3]:
import pandas as pd
def preprocess( directory , list_file , count = 1):
    data_frame = pd.DataFrame( { "Sentence" : [] , "Word" : [] , "Tag" : [] } )
    for file in list_file :
        word = read_all_line( directory , file ).split('|')
        pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0'] )
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame

In [4]:
#directory of corpus
corpus_directory = "corpus_directory"
corpus_file = ("Politic9.txt" , "Allcolumn.txt")
corpus_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" ,
                "result4.txt")
corpus_test = ("Result_c_2_1.txt" , "Result_c_2_2.txt")

In [5]:
data_frame = preprocess( corpus_directory , corpus_files )
data_frame
number_words = len( set( list( data_frame["Word"].values ) ) )
number_tags = len( list( set( data_frame["Tag"].values ) ) )
print( "Have type of tag is " + str( number_tags ) + 
        " tags from " + str(number_words) + " words")
# print(set( list( data_frame["Tag"].values )))

Have type of tag is 14 tags from 6966 words


In [6]:
data_frame

,Sentence,Word,Tag
0,Sentence: 1,"""",O
1,Sentence: 1,ทักษิณ,per
2,Sentence: 1,"""",O
3,Sentence: 1,,O
4,Sentence: 1,ขอ,O
...,...,...,...
67632,Sentence: 4010,แสดง,O
67633,Sentence: 4010,ความ,O
67634,Sentence: 4010,รู้สึก,O
67635,Sentence: 4010,ในทำนองเดียวกัน,O


Follow https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2

In [7]:
data_frame = data_frame.fillna(method='ffill')
data_frame['Sentence'].nunique(), data_frame.Word.nunique(), data_frame.Tag.nunique()

(4010, 6966, 14)

In [8]:
data_frame.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,,21
1,O,56284
2,loc,2077
3,loc_cont,39
4,loc_end,120
5,loc_start,120
6,org,2242
7,org_cont,1721
8,org_end,1020
9,org_start,1032


In [9]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [10]:
X = data_frame.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = data_frame.Tag.values

In [23]:
y

array(['O', 'per', 'O', ..., 'O', 'O', 'O'], dtype=object)

In [11]:
classes = np.unique(y)
classes = classes.tolist()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((45316, 10976), (45316,))

In [13]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


-- Epoch 1-- Epoch 1-- Epoch 1
-- Epoch 1


-- Epoch 1-- Epoch 1
-- Epoch 1
-- Epoch 1

Norm: 12.73, NNZs: 145, Bias: -2.000000, T: 45316, Avg. loss: 0.002670
Total training time: 2.02 seconds.
-- Epoch 1
Norm: 58.97, NNZs: 2052, Bias: -2.000000, T: 45316, Avg. loss: 0.025929Norm: 49.30, NNZs: 1416, Bias: -2.000000, T: 45316, Avg. loss: 0.018823
Total training time: 2.14 seconds.

Total training time: 2.07 seconds.
-- Epoch 1
-- Epoch 1
Norm: 42.99, NNZs: 1231, Bias: -4.000000, T: 45316, Avg. loss: 0.050997
Total training time: 2.22 seconds.


[Parallel(n_jobs=-1)]: Done   3 out of  14 | elapsed:    2.2s remaining:    8.0s
[Parallel(n_jobs=-1)]: Done   5 out of  14 | elapsed:    2.3s remaining:    4.2s


-- Epoch 1
Norm: 6.93, NNZs: 48, Bias: -3.000000, T: 45316, Avg. loss: 0.001125
Total training time: 2.18 seconds.
-- Epoch 1
Norm: 12.65, NNZs: 154, Bias: -3.000000, T: 45316, Avg. loss: 0.003134
Total training time: 2.23 seconds.
-- Epoch 1
Norm: 6.32, NNZs: 34, Bias: -3.000000, T: 45316, Avg. loss: 0.000552
Total training time: 2.37 seconds.


[Parallel(n_jobs=-1)]: Done   7 out of  14 | elapsed:    2.6s remaining:    2.6s


Norm: 112.14, NNZs: 5940, Bias: 1.000000, T: 45316, Avg. loss: 0.162945
Total training time: 2.92 seconds.
Norm: 40.96, NNZs: 1221, Bias: -2.000000, T: 45316, Avg. loss: 0.025068
Total training time: 1.88 seconds.
Norm: 38.60, NNZs: 1205, Bias: -2.000000, T: 45316, Avg. loss: 0.015359
Total training time: 1.72 seconds.
Norm: 30.76, NNZs: 742, Bias: -2.000000, T: 45316, Avg. loss: 0.009710
Total training time: 1.87 seconds.
Norm: 27.86, NNZs: 648, Bias: -4.000000, T: 45316, Avg. loss: 0.026613
Total training time: 1.79 seconds.
Norm: 38.60, NNZs: 1178, Bias: -2.000000, T: 45316, Avg. loss: 0.025951
Total training time: 1.92 seconds.
Norm: 38.70, NNZs: 1180, Bias: -2.000000, T: 45316, Avg. loss: 0.015866
Total training time: 1.72 seconds.


[Parallel(n_jobs=-1)]: Done   9 out of  14 | elapsed:    4.0s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  11 out of  14 | elapsed:    4.1s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    4.1s finished


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=5, n_iter_no_change=5, n_jobs=-1,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=10, warm_start=False)

In [14]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['',
 'O',
 'loc',
 'loc_cont',
 'loc_end',
 'loc_start',
 'org',
 'org_cont',
 'org_end',
 'org_start',
 'per',
 'per_cont',
 'per_end']

In [15]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

                   0.33      0.17      0.22         6
           O       0.90      0.99      0.94     18568
         loc       0.86      0.63      0.73       681
    loc_cont       0.00      0.00      0.00        14
     loc_end       0.00      0.00      0.00        37
   loc_start       0.08      0.03      0.04        35
         org       0.89      0.54      0.67       751
    org_cont       0.65      0.09      0.16       543
     org_end       0.35      0.41      0.38       323
   org_start       0.40      0.14      0.21       346
         per       0.62      0.27      0.38       181
    per_cont       0.00      0.00      0.00       303
     per_end       0.79      0.17      0.28       260

   micro avg       0.88      0.89      0.88     22048
   macro avg       0.45      0.27      0.31     22048
weighted avg       0.85      0.89      0.86     22048



/home/sevinar/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
